<a href="https://colab.research.google.com/github/Avelez164/Data-Science-Big-Data/blob/main/CPSC_375_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Datasets

1. Data on COVID-19 from Our World in Data
Their complete dataset contains a lot of information including the number of deaths, cases, vaccinations, hospitalizations, and several other country-specific pieces of information relevant to understanding the effects of COVID. Note that you can read the “raw” CSV file from a URL directly, like so:
pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")
2. Population estimates from The World Bank Group’s DataBank
